In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
jobs = pd.read_csv('../data/fullsample-corrected.csv')

jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'],
                               infer_datetime_format = True,
                               errors = 'coerce')

jobs['END'] = pd.to_datetime(jobs['END'],
                             infer_datetime_format = True,
                             errors = 'coerce')

jobs = jobs.dropna(subset = ['BEGIN', 'END'])

jobs.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,TIMEOUT,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:15
2,30858137,TIMEOUT,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:15
3,30935078,TIMEOUT,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:15
4,31364111_2,TIMEOUT,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:9
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0


In [4]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7395234 entries, 1 to 7395884
Data columns (total 12 columns):
 #   Column     Dtype         
---  ------     -----         
 0   JOBID      object        
 1   STATE      object        
 2   BEGIN      datetime64[ns]
 3   END        datetime64[ns]
 4   REQMEM     object        
 5   USEDMEM    object        
 6   REQTIME    object        
 7   USEDTIME   object        
 8   NODES      int64         
 9   CPUS       int64         
 10  PARTITION  object        
 11  EXITCODE   object        
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 733.5+ MB


In [5]:
jobs['JOBLENGTH'] = jobs['END'] - jobs['BEGIN']

jobs.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,JOBLENGTH
1,30853133,TIMEOUT,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:15,30 days 00:00:23
2,30858137,TIMEOUT,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:15,30 days 00:00:14
3,30935078,TIMEOUT,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:15,29 days 04:00:04
4,31364111_2,TIMEOUT,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:9,24 days 09:00:17
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0,20 days 08:32:27


In [6]:
jobs['MEMPERUNIT'] = jobs['REQMEM'].str.extract(r'(M\w)')
jobs['REQMEM'] = jobs['REQMEM'].str.extract(r'(\d+)')

In [7]:
jobs.loc[jobs['MEMPERUNIT'] == 'Mn', 'REQMEMTOT'] = jobs['REQMEM'] * jobs['NODES']
jobs.loc[jobs['MEMPERUNIT'] == 'MC', 'REQMEMTOT'] = jobs['REQMEM'] * jobs['CPUS']

In [8]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7395234 entries, 1 to 7395884
Data columns (total 15 columns):
 #   Column      Dtype          
---  ------      -----          
 0   JOBID       object         
 1   STATE       object         
 2   BEGIN       datetime64[ns] 
 3   END         datetime64[ns] 
 4   REQMEM      object         
 5   USEDMEM     object         
 6   REQTIME     object         
 7   USEDTIME    object         
 8   NODES       int64          
 9   CPUS        int64          
 10  PARTITION   object         
 11  EXITCODE    object         
 12  JOBLENGTH   timedelta64[ns]
 13  MEMPERUNIT  object         
 14  REQMEMTOT   object         
dtypes: datetime64[ns](2), int64(2), object(10), timedelta64[ns](1)
memory usage: 1.1+ GB


In [9]:
jobs.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,JOBLENGTH,MEMPERUNIT,REQMEMTOT
1,30853133,TIMEOUT,2021-08-06 11:36:09,2021-09-05 11:36:32,262144,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:15,30 days 00:00:23,Mn,262144
2,30858137,TIMEOUT,2021-08-06 19:04:39,2021-09-05 19:04:53,204800,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:15,30 days 00:00:14,Mn,204800
3,30935078,TIMEOUT,2021-08-09 16:52:51,2021-09-07 20:52:55,65536,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:15,29 days 04:00:04,Mn,65536
4,31364111_2,TIMEOUT,2021-08-17 07:45:07,2021-09-10 16:45:24,16384,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:9,24 days 09:00:17,Mn,16384
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0,20 days 08:32:27,Mn,16384


In [10]:
jobs.to_csv('../data/jobs-corrected_clean.csv',
           index = False)

In [2]:
logs = pd.read_csv('../data/slurm_both.csv')

logs.head()

,time,user,retry,elapsed_time,return_code,command,server
0,2020-10-16 08:15:39.278699,0,0,0.073476,0,sacct,ce5
1,2020-10-16 08:18:08.313309,0,0,0.183632,0,sacct,ce5
2,2020-10-16 08:22:48.128689,0,0,0.075471,0,sacct,ce5
3,2020-10-16 08:25:13.257408,0,0,0.094844,0,sacct,ce5
4,2020-10-16 08:31:01.460723,0,0,0.074988,0,sacct,ce5


In [23]:
logs.columns

Index(['time', 'user', 'retry', 'elapsed_time', 'return_code', 'command',
       'server'],
      dtype='object')

In [3]:
cols = {
    'time': 'TIME',
    'user': 'USER',
    'retry': 'RETRY',
    'elapsed_time': 'TIMELAPSE',
    'return_code': 'RETURNCODE',
    'command': 'COMMAND',
    'server': 'SERVER'
}

In [4]:
logs = logs.rename(columns = cols)

In [5]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9547413 entries, 0 to 9547412
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   TIME        object 
 1   USER        int64  
 2   RETRY       int64  
 3   TIMELAPSE   float64
 4   RETURNCODE  int64  
 5   COMMAND     object 
 6   SERVER      object 
dtypes: float64(1), int64(3), object(3)
memory usage: 509.9+ MB


In [11]:
logs['UNRESPONSIVE'] = ((logs['RETURNCODE'] == 1) &
                      (logs['TIMELAPSE'] > 15.0) &
                      (logs['COMMAND'] == 'sbatch'))

In [12]:
logs['UNRESPONSIVE'].value_counts()

False    9440043
True      107370
Name: UNRESPONSIVE, dtype: int64

In [8]:
logs['TESTING'] = logs['USER'] == 9204

In [9]:
logs['TESTING'].value_counts()

False    9232704
True      314709
Name: TESTING, dtype: int64

In [13]:
logs.to_csv('../data/logs_clean.csv',
           index = False)